<a href="https://colab.research.google.com/github/hardik1408/CS771/blob/hardik/emoticon_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_emoticon_df = pd.read_csv("train_emoticon.csv")
train_emoticon_X = train_emoticon_df['input_emoticon'].tolist()
train_emoticon_Y = train_emoticon_df['label'].tolist()

In [19]:
valid_df = pd.read_csv("valid_emoticon.csv")
valid_X = valid_df['input_emoticon'].tolist()
valid_Y = valid_df['label'].tolist()

In [46]:
len(valid_X)

489

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [26]:
tokenizer = Tokenizer(char_level=True)  # char_level=True because emojis are treated like characters
tokenizer.fit_on_texts(train_emoticon_X)
emoji_sequences = tokenizer.texts_to_sequences(train_emoticon_X)

In [28]:
tokenizer = Tokenizer(char_level=True)  # char_level=True because emojis are treated like characters
tokenizer.fit_on_texts(valid_X)
emoji_sequences_valid = tokenizer.texts_to_sequences(valid_X)

In [29]:
max_len = max([len(seq) for seq in emoji_sequences])
emoji_sequences_padded_valid = pad_sequences(emoji_sequences, maxlen=max_len, padding='post')

In [27]:
max_len = max([len(seq) for seq in emoji_sequences])
emoji_sequences_padded = pad_sequences(emoji_sequences, maxlen=max_len, padding='post')
X_train = emoji_sequences_padded
y_train = train_emoticon_Y
y_train = np.array(y_train)
# # Step 4: Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(emoji_sequences_padded, train_emoticon_Y, test_size=0.1, random_state=42)

In [30]:
X_test = emoji_sequences_padded_valid
y_test = valid_Y
y_test = np.array(y_test)

In [41]:
X_train.size

92040

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Step 1: Tokenize emoji sequences
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(train_emoticon_X)
X = tokenizer.texts_to_sequences(train_emoticon_X)

# Step 2: Pad sequences
max_len = max([len(seq) for seq in X])
X_padded = pad_sequences(X, maxlen=max_len, padding='post')

# Step 3: Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, train_emoticon_Y, test_size=0.2, random_state=42)
y_test = np.array(y_test)
y_train = np.array(y_train)
# Step 4: Define a CNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8, input_length=max_len))
model.add(Conv1D(32, 3, activation='relu'))  # 1D Convolutional Layer
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())  # Flatten for Dense Layers
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary Classification

# Step 5: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

# Step 7: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')


Epoch 1/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5091 - loss: 0.6918 - val_accuracy: 0.8249 - val_loss: 0.6158
Epoch 2/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8377 - loss: 0.4807 - val_accuracy: 0.9082 - val_loss: 0.2449
Epoch 3/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9129 - loss: 0.2447 - val_accuracy: 0.9131 - val_loss: 0.2286
Epoch 4/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9241 - loss: 0.2139 - val_accuracy: 0.9082 - val_loss: 0.2208
Epoch 5/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9268 - loss: 0.2066 - val_accuracy: 0.9103 - val_loss: 0.2157
Epoch 6/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9334 - loss: 0.1949 - val_accuracy: 0.9068 - val_loss: 0.2148
Epoch 7/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9401 - loss: 0.1860 - val_accuracy: 0.9089 - val_loss: 0.2132
Epoch 8/30
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9438 - loss: 0.1727 - val_accuracy: 0.

In [50]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load the CSV files
train_df = pd.read_csv('train_emoticon.csv')
validation_df = pd.read_csv('valid_emoticon.csv')

# Assuming the CSV files have 'emojis' and 'label' columns
train_texts = train_df['input_emoticon'].values
train_labels = train_df['label'].values

validation_texts = validation_df['input_emoticon'].values
validation_labels = validation_df['label'].values

# Step 2: Tokenize and pad the emoji sequences
tokenizer = Tokenizer(char_level=True)  # Tokenizing each emoji as a character
tokenizer.fit_on_texts(train_texts)  # Fit only on training data

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
validation_sequences = tokenizer.texts_to_sequences(validation_texts)

# Pad the sequences to the same length
max_len = max([len(seq) for seq in train_sequences])  # Maximum sequence length in train data
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
validation_padded = pad_sequences(validation_sequences, maxlen=max_len, padding='post')

# Convert labels into numeric format if they are not (if necessary)
train_labels = train_labels.astype(int)
validation_labels = validation_labels.astype(int)


In [51]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8, input_length=max_len))  # Embedding layer
model.add(LSTM(16))  # LSTM layer with 16 units
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [53]:
# Train the model using the training data and validate on the validation set
model.fit(train_padded, train_labels, epochs=30, validation_data=(validation_padded, validation_labels))
# Evaluate the model on the validation data
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation Accuracy: {accuracy}')


Epoch 1/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9450 - loss: 0.1328 - val_accuracy: 0.9387 - val_loss: 0.1429
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9520 - loss: 0.1197 - val_accuracy: 0.9284 - val_loss: 0.1512
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9556 - loss: 0.1115 - val_accuracy: 0.9407 - val_loss: 0.1292
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9556 - loss: 0.1123 - val_accuracy: 0.9305 - val_loss: 0.1426
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9644 - loss: 0.0991 - val_accuracy: 0.9387 - val_loss: 0.1372
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9608 - loss: 0.0990 - val_accuracy: 0.9366 - val_loss: 0.1310
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9623 - loss: 0.0958 - val_accuracy: 0.9489 - val_loss: 0.1233
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9573 - loss: 0.1029 - val_accuracy: 

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization

# Define the improved LSTM model
model = Sequential()

# Embedding layer: increases embedding dimension size for richer representation
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_len))

# First LSTM layer: Bidirectional LSTM for capturing dependencies in both directions
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Return sequences for stacking another LSTM layer
model.add(Dropout(0.3))  # Dropout to prevent overfitting

# Second LSTM layer: another Bidirectional LSTM layer for deeper sequential learning
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))  # Another dropout layer

# Batch Normalization: helps with stabilizing and accelerating training
model.add(BatchNormalization())

# Dense Layer: fully connected layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))  # Dropout for regularization

# Output Layer: Sigmoid for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer and learning rate adjustment
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary for reference
model.summary()

# Train the model using the train set and validate on the validation set
model.fit(train_padded, train_labels, epochs=30, batch_size=32, validation_data=(validation_padded, validation_labels))


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.6411 - loss: 0.5930 - val_accuracy: 0.8425 - val_loss: 0.5474
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.8740 - loss: 0.2928 - val_accuracy: 0.9202 - val_loss: 0.3089
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.8917 - loss: 0.2564 - val_accuracy: 0.9018 - val_loss: 0.2336
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - accuracy: 0.8975 - loss: 0.2348 - val_accuracy: 0.9305 - val_loss: 0.1961
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.8907 - loss: 0.2382 - val_accuracy: 0.8834 - val_loss: 0.2439
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.9172 - loss: 0.1937 - val_accuracy: 0.9182 - val_loss: 0.1680
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.9325 - loss: 0.1583 - val_accuracy: 0.9141 - val_loss: 0.1612
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.9444 - loss: 0.1401 - val

In [58]:
# Evaluate the model on the validation data
loss, accuracy = model.evaluate(validation_padded, validation_labels)

# Print the final accuracy
print(f'Validation Accuracy: {accuracy:.4f}')


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9729 - loss: 0.0868
Validation Accuracy: 0.9673
